# Загрузка

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
!pip install -r ../requirements_yolo.txt

In [ ]:
!pip install -r ../requirements_detectron.txt
!pip install --no-build-isolation git+https://github.com/facebookresearch/detectron2.git

In [ ]:
from license_plate_detection.dataset import unzip_dataset, process_coco_dataset, download_dataset
from license_plate_detection.config import * # or from license_plate_detection.colab_config import * for colab

In [ ]:
@title Колаб

# from google.colab import drive
# drive.mount('/content/drive')

%cd /content
!rm -rf /content/license-plate-detection
!git clone https://github.com/ArconRM/license-plate-detection
%cd /content/license-plate-detection

In [ ]:
@title Локально

download_dataset(DATASET_URL, DATASET_PATH) # skip if already downloaded zip to DATASET_PATH
unzip_dataset(DATASET_PATH, RAW_DATA_DIR)

In [ ]:
splits = process_coco_dataset(
    in_base = RAW_DATA_DIR,
    out_base=PROCESSED_DATA_DIR,
    train_ratio=0.7,
    val_ratio=0.2,
    seed=42
)

# YOLO

In [ ]:
from license_plate_detection.yolo_utils import convert_coco_to_yolo
from license_plate_detection.train import train_yolo_model

In [ ]:
convert_coco_to_yolo(PROCESSED_DATA_DIR, YOLO_DATA_DIR)

In [ ]:
train_yolo_model('yolo5s.pt', 'data.yaml', epochs=20, img_size=768)

In [ ]:
train_yolo_model('yolo8s.pt', 'data.yaml', epochs=20, img_size=768)

In [ ]:
train_yolo_model('yolo11s.pt', 'data.yaml', epochs=20, img_size=768)

# Detectron

In [ ]:
from detectron2.data.datasets import register_coco_instances

from license_plate_detection.train import train_detectron
from license_plate_detection.predict import predict_detectron

In [ ]:
register_coco_instances("plate_train", {}, "/content/license-plate-detection/data/processed/train.json", "/content/license-plate-detection/data/processed/images/train")
register_coco_instances("plate_val", {}, "/content/license-plate-detection/data/processed/val.json", "/content/license-plate-detection/data/processed/images/val")
register_coco_instances("plate_test", {}, "/content/license-plate-detection/data/processed/test.json", "/content/license-plate-detection/data/processed/images/test")

In [ ]:
train_detectron(
    train_dataset_name="plate_train",
    val_dataset_name="plate_val",
    num_classes=1,
    batch_size=3,
    max_iter=20000,
    output_dir="./output"
)

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output